In [1]:
import numpy as np
import pandas as pd
import random

As mentioned in read me 
Default number of hidden states is 5
Default number is also 5, all the elements in visible sequence is between [0,4]
If you have raw sequence on which HMM has to be applied first get number number of unique hidden states and unique visible states and map states and visible states as follows

In [2]:
sequence=[0,1,2,3,1]
sequence=np.array(sequence)
intial=[0.05,0.5,0.3,0.05,0.1]
intial=np.array(intial)

In [59]:
class HMM1:
    def __init__(self,sequence,initial_matrix,number_states=5,number_output=5,n_epochs=30):
        self.seq=sequence
        self.tras=np.random.uniform(0,1,(number_states,number_states))  ## Transition Matrix Initialized with random number drawn from uniform (0,1)
        self.emiss=np.random.uniform(0,1,(number_states,number_output)) ## Emission Matrix

        self.iprob=initial_matrix         ## Intial Probabilty matrix (Pie)
        
        self.states=number_states         ## Number of hidden States
        
        self.out=number_output            ## Number of Visible States
        
        self.epochs=n_epochs              ## Number of Iterations default value is 30
        
        self.BDP=np.zeros(shape=(number_states,len(self.seq)))  ## 2d numpy array to store alpha(state,time)
        
        self.FDP=np.zeros(shape=(number_states,len(self.seq)))   ## 2d numpy array to store  Beta(state,time)
        
        self.n_epochs=n_epochs
        
    def forward(self):
        nstates=self.states
        nlength=len(self.seq)
        
        for time in range(nlength):
            for state in range(nstates):
                if(time==0):
                    self.BDP[state][time]=self.iprob[state]*self.emiss[state][self.seq[time]]
                else:
                    for pre_state in range(nstates):
                        self.BDP[state][time]+=self.BDP[pre_state][time-1]*self.tras[pre_state][state]*self.emiss[state][self.seq[time]]
                        
                        
                        
    def decoding(self):
        nstates=self.states
        nlength=len(self.seq)
        
        deco=np.zeros(shape=(nstates,nlength))
        out=[]
        
        
        for time in range(nlength):
            for state in range(nstates):
                if(time==0):
                    deco[state][time]=self.iprob[state]*self.emiss[state][self.seq[time]]
                else:
                    maxi=0.0
                    for prev_state in range(nstates):
                        maxi=max(maxi,deco[prev_state][time-1]*self.tras[prev_state][state]*self.emiss[state][self.seq[time]])
                    deco[state][time]=maxi
            
            maxi=0.0
            index=0
            
            for state in range(nstates):
                if deco[state][time]>maxi:
                    index=state
                    maxi=deco[state][time]
            
            out.append(index)
        out=np.array(out)
        
        return out
    
    def beta(self):
        nstates=self.states
        nlength=len(self.seq)
        
        for time in range(nlength-1,-1,-1):
            for state in range(nstates):
                if(time==nlength-1):
                    self.FDP[state][time]=1.0
                else:
                    for nxt_state in range(nstates):
                        self.FDP[state][time]+=self.FDP[nxt_state][time+1]*self.tras[state][nxt_state]*self.emiss[nxt_state][self.seq[time+1]]
        sum=0
        
        for state in range(nstates):
            sum+=self.FDP[state][0]*self.iprob[state]*self.tras[state][self.seq[0]]
        
        return sum
    
    def not_quite_epsilon(self,time,pre_state,nxt_state):
        return self.tras[pre_state][nxt_state]*self.BDP[pre_state][time]*self.emiss[nxt_state][self.seq[time+1]]*self.FDP[nxt_state][time+1]
    
    def epsilon(self,time,pre_state,nxt_state):
        
        nstates=self.states
        nlength=len(self.seq)
        deno=0
        
        for states in range(nstates):
            deno+=self.BDP[states][time]*self.FDP[states][time]
            
        if(deno==0):
            print( self.not_quite_epsilon(time,pre_state,nxt_state))
        return self.not_quite_epsilon(time,pre_state,nxt_state)/deno
    
    def gamma(self,time,state):
        deno=0
        nstates=self.states
        for states in range(nstates):
            deno+=self.BDP[states][time]*self.FDP[states][time]
            
        return self.BDP[state][time]*self.FDP[state][time]/deno
            
        
    def transition_correction(self):
        nstates=self.states
        nlength=len(self.seq)
        
        for curr_state in range(nstates):
            for nxt_state in range(nstates):
                num=0
                den=0
                for time in range(nlength-1):
                   
                    num+=self.epsilon(time,curr_state,nxt_state)
                    for state in range(nstates):
                        den+=self.epsilon(time,curr_state,state)
                    
                if(den==0):
                    print("ok")
                #self.tras[curr_state][nxt_state]=num/deno
             
                
    def emission_correction(self):
        nstates=self.states
        nlength=len(self.seq)
        output=self.out
        
        for state in range(nstates):
            for out in range(output):
                num=0
                den=0
                for time in range(nlength):
                    if(self.seq[time]==out):
                        num+=self.gamma(time,state)
                    den+=self.gamma(time,state)
                self.emiss[state][out]=num/den
    
    def give_prob(self):
        self.forward()
        nstates=self.states
        nlength=len(self.seq)
        sum=0
        for states in range(nstates):
            sum+=self.BDP[states][nlength-1]
        return sum
    def display(self):
        print(self.tras)
    def forward_backward(self):
        for epoch in range(self.epochs):
            self.forward()
            self.beta()
            self.transition_correction()
            self.emission_correction()

In [60]:
k=HMM1(sequence,intial)

In [61]:
k.forward()

In [62]:
x=k.decoding()

In [63]:
x

array([1, 2, 0, 2, 2])

In [64]:
m=k.give_prob()

In [65]:
k.display()

[[0.62068193 0.42003623 0.74205509 0.7374311  0.67727546]
 [0.89785361 0.74692186 0.87306654 0.31579948 0.70154287]
 [0.51666311 0.10980215 0.67963944 0.99617427 0.44352027]
 [0.25454176 0.76718435 0.7022636  0.59303444 0.81955061]
 [0.46335154 0.66351785 0.28998859 0.90195513 0.91695913]]


In [66]:
k.transition_correction()

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


C:\Users\GauravS21\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:98: RuntimeWarning: invalid value encountered in double_scalars
